In [15]:
import pandas as pd
import plotly.express as px

data_file = 'data/2023.xlsx'

df = pd.ExcelFile(data_file)
sheet_names = df.sheet_names
print(sheet_names)

['Male', 'Female']


In [16]:
df_m = pd.read_excel(data_file, sheet_name='Male',header=None)
df_f = pd.read_excel(data_file, sheet_name='Female',header=None)
df_m.head()

,0,1,2
0,Hawaii,1.0,32.5
1,New York,2.0,32.2
2,California,3.0,32.1
3,Massachusetts,4.0,32.0
4,Connecticut,5.0,31.8


In [17]:
df_m.columns = ['State','M_Rank','M_MedianAge']
df_f.columns = ['State','F_Rank','F_MedianAge']
df_m.head()

,State,M_Rank,M_MedianAge
0,Hawaii,1.0,32.5
1,New York,2.0,32.2
2,California,3.0,32.1
3,Massachusetts,4.0,32.0
4,Connecticut,5.0,31.8


## Male

In [18]:
# 创建条形图显示各州的中位数年龄
fig = px.bar(df_m, x='State', y='M_MedianAge', 
             title='Median Age at First Marriage by State (Males)',
             color='M_MedianAge',
             labels={'M_MedianAge':'Median Age'})

# 为USA添加竖虚线
fig.add_vline(
    x=df_m[df_m['State'] == 'USA'].index[0],
    line_width=2,
    line_dash="dash",
    line_color="#d63031"
)

fig.show()

## Female

In [19]:
# 创建条形图显示各州的中位数年龄
fig = px.bar(df_f, x='State', y='F_MedianAge', 
             title='Median Age at First Marriage by State (Females)',
             color='F_MedianAge',
             labels={'F_MedianAge':'Median Age'})

# 为USA添加竖虚线
fig.add_vline(
    x=df_f[df_f['State'] == 'USA'].index[0],
    line_width=2,
    line_dash="dash",
    line_color="#d63031"
)

fig.show()

## Female vs Male

In [20]:
df_all = pd.merge(df_m, df_f, on='State')
df_all.head()

,State,M_Rank,M_MedianAge,F_Rank,F_MedianAge
0,Hawaii,1.0,32.5,36.0,27.1
1,New York,2.0,32.2,2.0,30.5
2,California,3.0,32.1,7.0,30.2
3,Massachusetts,4.0,32.0,2.0,30.5
4,Connecticut,5.0,31.8,4.0,30.3


In [21]:
df_all['MedianAge_diff'] = round(df_all['M_MedianAge'] - df_all['F_MedianAge'],2)
df_all.head()

,State,M_Rank,M_MedianAge,F_Rank,F_MedianAge,MedianAge_diff
0,Hawaii,1.0,32.5,36.0,27.1,5.4
1,New York,2.0,32.2,2.0,30.5,1.7
2,California,3.0,32.1,7.0,30.2,1.9
3,Massachusetts,4.0,32.0,2.0,30.5,1.5
4,Connecticut,5.0,31.8,4.0,30.3,1.5


In [22]:
print(df_all)

                   State  M_Rank  M_MedianAge  F_Rank  F_MedianAge  \
0                 Hawaii     1.0         32.5    36.0         27.1   
1               New York     2.0         32.2     2.0         30.5   
2             California     3.0         32.1     7.0         30.2   
3          Massachusetts     4.0         32.0     2.0         30.5   
4            Connecticut     5.0         31.8     4.0         30.3   
5           Rhode Island     6.0         31.6     8.0         30.1   
6   District of Columbia     6.0         31.6     1.0         31.7   
7               Maryland     8.0         31.5     9.0         29.6   
8             New Jersey     8.0         31.5     4.0         30.3   
9               Delaware    10.0         31.2     9.0         29.6   
10               Vermont    10.0         31.2     4.0         30.3   
11               Arizona    12.0         31.1    19.0         28.5   
12          Pennsylvania    12.0         31.1    12.0         29.4   
13               Flo

In [23]:
# 创建条形图显示各州的中位数年龄
fig = px.bar(
    df_all,
    x="State",
    y="MedianAge_diff",
    title="Median Age at First Marriage by State (MedianAge_diff(M-F))",
    color="MedianAge_diff",
    labels={"MedianAge_diff": "Median Age Diff"},
)
# 为USA添加竖虚线
fig.add_vline(
    x=df_all[df_all['State'] == 'USA'].index[0],
    line_width=2,
    line_dash="dash",
    line_color="#d63031"
)

fig.show()

## Map

plotly 的 choropleth, location 参数接收 Sate Code, 引入 state_code 数据框, 合并 df_all 和 state_code 数据框, 并根据 Alpha code 绘制地图

In [24]:
state_code = pd.read_csv('data/usa_state_code.csv')
state_code.head()

,ode,State,Abbreviation,Alpha code
0,1,Alabama,Ala.,AL
1,2,Alaska,NaN,AK
2,4,Arizona,Ariz.,AZ
3,5,Arkansas,Ark.,AR
4,6,California,Calif.,CA


In [25]:
df_all = pd.merge(df_all, state_code, on='State')
df_all.head()

,State,M_Rank,M_MedianAge,F_Rank,F_MedianAge,MedianAge_diff,ode,Abbreviation,Alpha code
0,Hawaii,1.0,32.5,36.0,27.1,5.4,15,NaN,HI
1,New York,2.0,32.2,2.0,30.5,1.7,36,N.Y.,NY
2,California,3.0,32.1,7.0,30.2,1.9,6,Calif.,CA
3,Massachusetts,4.0,32.0,2.0,30.5,1.5,25,Mass.,MA
4,Connecticut,5.0,31.8,4.0,30.3,1.5,9,Conn.,CT


#### male

In [26]:
fig = px.choropleth(
    df_all,
    locations="Alpha code",  # 州名列
    locationmode="USA-states",  # 设置为美国州级地图
    color="M_MedianAge",  # 根据中位数年龄着色
    scope="usa",  # 限定美国地图范围
    title='Median Age at First Marriage by State',
    color_continuous_scale="Viridis",  # 颜色方案
    hover_data=["State", "M_Rank", "F_Rank", "M_MedianAge", "F_MedianAge"],
)

fig.update_layout(
    showlegend=True,
    margin=dict(l=0, r=0, t=50, b=10),
    autosize=False,  # 禁用自动调整大小
)

fig.show()

#### female

In [27]:
# 假设您的数据框df_m包含'State'列和'MedianAge'列
fig = px.choropleth(
    df_all,
    locations="Alpha code",  # 州名列
    locationmode="USA-states",  # 设置为美国州级地图
    color="F_MedianAge",  # 根据中位数年龄着色
    scope="usa",  # 限定美国地图范围
    title='Median Age at First Marriage by State',
    color_continuous_scale="Viridis",  # 颜色方案
    hover_data=['State',"M_Rank", "F_Rank", "M_MedianAge", "F_MedianAge",'MedianAge_diff'],
)

fig.update_traces(
    hovertemplate="<b>%{customdata[0]}</b><br>"
    + "Male Rank: %{customdata[1]}<br>"
    + "Female Rank: %{customdata[2]}<br>"
    + "Male Median Age: %{customdata[3]}<br>"
    + "Female Median Age: %{customdata[4]}<br>"
    + "Difference: %{customdata[5]}<br>"
    + "<extra></extra>"
)

fig.update_layout(
    showlegend=True,
    margin=dict(l=0, r=0, t=50, b=0),
    #    width=1000,  # 设置宽度
    # height=600,  # 设置高度
    autosize=False  # 禁用自动调整大小
)

fig.show()